# 연합인포맥스 기사 탐색

In [26]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup as bsp
import matplotlib
import matplotlib.pyplot as plt
import sqlite3
import re
from datetime import datetime, timedelta,date
import sys

In [27]:
def makeKey(txt=''):
    txt_=re.sub('http\S*//',' ',txt)
    key=re.sub('[^a-zA-Z0-9%]','',txt_)
    return key

In [28]:
def getData(whr=''):
    dbname='yinfo_se.db'
    with sqlite3.connect(dbname) as conn:
        cursor=conn.cursor()
        sql='select * from news '+whr
        res=cursor.execute(sql).fetchall()
    return(res)

In [29]:
# 기존 dB에 추가
def insert_db(data,dbname):
    dbname= dbname + '.db'
    with sqlite3.connect(dbname) as conn:
        cursor=conn.cursor()
        sql='select * from news where key="'+data[3]+'"'
        res=cursor.execute(sql).fetchall()
        if(len(res)>0):
            print("이미 존재 합니다.:"+data[1])
        else:
            sql='insert into news (title,date,url,key,content) values (?,?,?,?,?)'
            res=cursor.execute(sql,data)
            print('.',end='')

In [30]:
# find last Page
def Flp(sdate,edate,compn):
    infourl_ = 'https://news.einfomax.co.kr/news/articleList.html?&sc_area=A&view_type=sm&sc_section_code=&sc_area=T&sc_level=&sc_article_type=&sc_sdate={0}&sc_edate={1}&sc_order_by=E&sc_word={2}&sc_andor=OR&sc_word2='
    infourl = infourl_.format(sdate,edate,compn)
    ress = requests.get(infourl)
    srcc=ress.content.strip()
    htmls = bsp(srcc,'html.parser')
    if
    arti_ = htmls.select('ul.pagination li.pagination-end')[0]
    # last_arti = arti_
    lp = arti_.select('a')[0]['href']
    lastpage = lp.replace('/news/articleList.html?page=','').split("&")[0]
    return lastpage

In [31]:
## 인포맥스 작업 _ 개별 기사 사이트에서 직접 크롤링
def Eisub(turl):
    ress = requests.get(turl)
    srcc=ress.content.strip()
    htmls = bsp(srcc,'html.parser')
    title = htmls.select('.article-head-title')[0].text
    wday_ = htmls.select('.info-text li')[1].text
    date = re.sub('승인','',wday_)
    artia__ = htmls.select('#article-view-content-div')[0].text
    artia_ = re.sub('[\t\r\f\v\n]','',artia__)
    artia = artia_.split('(끝)')[0]              
    content = re.sub("\([가-힣]*=연합인포맥스\)","",artia)
    content = re.sub("(\s[가-힣]*\s기자\s)=","",content)
    content = re.sub("(\w*@\w*.\w*.\w*)",'',content)
    content = re.sub("(\s{3,10})",'',content)
    url = turl
    key = makeKey(url)
    data = [title,date,url,key,content]
    insert_db(data,dbn)
#     lii.append(result)
#     return (title, day, content, reporter)
# insert_db(data,'hk_news')

In [32]:
## 인포맥스 작업_ 기사리스트 크롤링
def Einfo(page,sdate,edate,compn):
    infourl_ = 'https://news.einfomax.co.kr/news/articleList.html?page={0}&sc_area=A&view_type=sm&sc_section_code=&sc_area=T&sc_level=&sc_article_type=&sc_sdate={1}&sc_edate={2}&sc_order_by=E&sc_word={3}&sc_andor=OR&sc_word2='
    infourl = infourl_.format(page,sdate,edate,compn)
    ress = requests.get(infourl)
    srcc=ress.content.strip()
    htmls = bsp(srcc,'html.parser')
    arti_ = htmls.select('section.article-list-content .list-block')
    for i in range(len(arti_)):
        arti = htmls.select('section.article-list-content .list-block')[i]
        turl_ = arti.select('.list-titles a')[0]['href']
        turl = 'https://news.einfomax.co.kr/' + turl_
        Eisub(turl)
#     return

In [33]:
## 작동 함수
def yinfo():
    dbn = 'yinfo_'+compn
    sdate = '2021-01-01'
    edate = '2021-03-31'
    LastP = int(Flp(sdate,edate,compn))
    for i in range(1,LastP+1):
        Einfo(i,sdate,edate,compn)

In [49]:
comlist = """삼성전자
SK하이닉스
네이버
LG화학
현대차
삼성바이오로직스
카카오
삼성SDI
기아차
포스코
현대모비스
LG전자
LG생활건강
삼성물산
KB금융
SK텔레콤
엔씨소프트
SK이노베이션
신한지주
삼성생명
한국전력
삼성SDS
아모레퍼시픽
삼성전기
하나금융지주
KT&G
롯데케미칼
한온시스템
S-Oil
삼성화재
LG디스플레이
고려아연
금호석유
현대글로비스
KT
기업은행
CJ제일제당
미래에셋대우
현대제철
강원랜드
LG유플러스
아모레G
오리온
LG이노텍
이마트
대한항공
한국금융지주
현대건설
코웨이
녹십자
두산중공업
유한양행
SKC
삼성중공업
한미사이언스
삼성카드
CJ대한통운
한미약품
팬오션
한국항공우주
롯데쇼핑
GS건설
동서
롯데지주
삼성증권
쌍용양회
호텔신라
만도
SK케미칼
DB손해보험
NH투자증권
에스원
GS리테일
한국가스공사
일진머티리얼즈
BGF리테일
삼성엔지니어링
신세계
OCI
대우건설
한화생명
하이트진로
한솔케미칼
DB하이텍
한샘
제일기획
현대미포조선
한화
현대로템
현대위아
BNK금융지주
대한유화
오뚜기
현대해상
LS
두산인프라코어
현대백화점
영원무역
현대엘리베이
KCC
효성
녹십자홀딩스
농심
종근당
아이에스동서
코오롱인더
부광약품
대웅제약
롯데정밀화학
SK네트웍스
한국콜마
한전KPS
동원시스템즈
코스맥스
한올바이오파마
이노션
영진약품
보령제약
세방전지
금호타이어
동국제강
LG상사
롯데칠성
영풍
GKL
태광산업
현대그린푸드
CJ CGV
후성
휴켐스
한섬
현대홈쇼핑
한세실업
풍산
오리온홀딩스
대상
LIG넥스원
한전기술
두산
락앤락
삼양홀딩스
동원F&B
넥센타이어
LG하우시스
일양약품
JW중외제약
SPC삼립
삼양사
빙그레"""

comlist = comlist.split("\n")

In [53]:
len(comlist)

149

In [52]:
#### 실행

for i in comlist:
    compn = i
    dbn = 'yinfo_'+compn
    print(compn)
    try:
        yinfo()          
    except:
        print(i*30)
        print("오류가 났습니다!!!!!!!!!!!!!!!!!")
        pass

삼성전자
이미 존재 합니다.:  2021.03.30 15:32
이미 존재 합니다.:  2021.03.26 15:55
이미 존재 합니다.:  2021.03.25 11:03
이미 존재 합니다.:  2021.03.25 08:54
이미 존재 합니다.:  2021.03.24 09:55
이미 존재 합니다.:  2021.03.23 08:13
이미 존재 합니다.:  2021.03.22 10:17
이미 존재 합니다.:  2021.03.22 08:56
이미 존재 합니다.:  2021.03.19 10:36
이미 존재 합니다.:  2021.03.17 23:03
이미 존재 합니다.:  2021.03.17 11:55
이미 존재 합니다.:  2021.03.17 11:15
이미 존재 합니다.:  2021.03.17 10:52
이미 존재 합니다.:  2021.03.17 10:23
이미 존재 합니다.:  2021.03.16 08:18
이미 존재 합니다.:  2021.03.15 18:37
이미 존재 합니다.:  2021.03.12 11:28
이미 존재 합니다.:  2021.03.12 08:58
이미 존재 합니다.:  2021.03.11 11:03
이미 존재 합니다.:  2021.03.11 09:52
이미 존재 합니다.:  2021.03.10 08:57
이미 존재 합니다.:  2021.03.09 18:54
이미 존재 합니다.:  2021.03.09 15:11
이미 존재 합니다.:  2021.03.09 10:30
이미 존재 합니다.:  2021.03.09 08:52
이미 존재 합니다.:  2021.03.07 10:24
이미 존재 합니다.:  2021.03.05 08:58
이미 존재 합니다.:  2021.03.04 11:21
이미 존재 합니다.:  2021.03.04 11:03
이미 존재 합니다.:  2021.03.04 09:41
이미 존재 합니다.:  2021.03.03 10:18
이미 존재 합니다.:  2021.02.26 11:24
이미 존재 합니다.:  2021.02.25 10:32
이미 존재

.......................OCI
.......대우건설
............한화생명
..................하이트진로
......한솔케미칼
한솔케미칼한솔케미칼한솔케미칼한솔케미칼한솔케미칼한솔케미칼한솔케미칼한솔케미칼한솔케미칼한솔케미칼한솔케미칼한솔케미칼한솔케미칼한솔케미칼한솔케미칼한솔케미칼한솔케미칼한솔케미칼한솔케미칼한솔케미칼한솔케미칼한솔케미칼한솔케미칼한솔케미칼한솔케미칼한솔케미칼한솔케미칼한솔케미칼한솔케미칼한솔케미칼
오류가 났습니다!!!!!!!!!!!!!!!!!
DB하이텍
DB하이텍DB하이텍DB하이텍DB하이텍DB하이텍DB하이텍DB하이텍DB하이텍DB하이텍DB하이텍DB하이텍DB하이텍DB하이텍DB하이텍DB하이텍DB하이텍DB하이텍DB하이텍DB하이텍DB하이텍DB하이텍DB하이텍DB하이텍DB하이텍DB하이텍DB하이텍DB하이텍DB하이텍DB하이텍DB하이텍
오류가 났습니다!!!!!!!!!!!!!!!!!
한샘
.제일기획
제일기획제일기획제일기획제일기획제일기획제일기획제일기획제일기획제일기획제일기획제일기획제일기획제일기획제일기획제일기획제일기획제일기획제일기획제일기획제일기획제일기획제일기획제일기획제일기획제일기획제일기획제일기획제일기획제일기획제일기획
오류가 났습니다!!!!!!!!!!!!!!!!!
현대미포조선
..한화
.....................................................................................현대로템
....현대위아
....BNK금융지주
BNK금융지주BNK금융지주BNK금융지주BNK금융지주BNK금융지주BNK금융지주BNK금융지주BNK금융지주BNK금융지주BNK금융지주BNK금융지주BNK금융지주BNK금융지주BNK금융지주BNK금융지주BNK금융지주BNK금융지주BNK금융지주BNK금융지주BNK금융지주BNK금융지주BNK금융지주BNK금융지주BNK금융지주BNK금융지주BNK금융지주BNK금융지주BNK금융지주BNK금융지주BNK금융지주
오류가 났습니다!!!!!!!!!!!!!!!!!
대한유화
.오뚜기
..현대해상
......LS
......

In [310]:
for i in k_list:
    compn_ = i
    print(compn_)

NAVER
LG화학
현대차
삼성바이오로직스
카카오
삼성SDI
기아차
POSCO
현대모비스
LG전자
LG생활건강
삼성물산
KB금융
SK텔레콤
엔씨소프트
SK이노베이션
신한지주
SK
삼성생명


In [ ]:
#데이터가 없는거 아모레G, 유한양행, 영원무역, 태광산업, 후성, 한세실업, 일양약품, JW중외제약

#회사 name때문에 결정되야할 것 : gs,sk,lg, cj, kt(수작업), kt&g,